# Counting Boats from Space - Part 3

In [ ]:
%reload_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv
%matplotlib inline

## 1. Install + import requirements / dependencies

In [ ]:
import sys
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install torchvision
!{sys.executable} -m pip install plotly

In [ ]:
# xcube_sh imports
from xcube_sh.cube import open_cube
from xcube_sh.observers import Observers

# Various utilities
import os
import shapely.geometry
import IPython.display
import numpy as np
import json
import torch

In [ ]:
from src.dataset import plot_geoloc
from src.GIS_utils import bbox_from_point
from src.config import CubeConfig
from src.preprocess import cube2tensor
from src.model import Model

In [ ]:
torch.cuda.is_available() # gpu support

In [ ]:
# constants
RADIUS = 2500 # AOI radius in meters
NDWI_threshold = 0.4 # NDWI threshold for water/land segmentation

## 2. Specify AOI + time window

In [ ]:
data_dir = "/home/jovyan/data" # data directory (path)
checkpoint_dir = "/home/jovyan/checkpoints"

In [ ]:
##### TODO: Add new locations to aoi.json

In [ ]:
with open(os.path.join(data_dir, 'aoi.json'), 'r') as f:
    aoi_file = json.load(f)

In [ ]:
plot_geoloc(list(aoi_file['Ports'].values()), [lat_lon for snaps in list(aoi_file['Straits'].values()) for lat_lon in snaps])

In [ ]:
#coords = aoi_file['Ports']['IT - Messina']
coords = aoi_file['Straits']['Dardanelles'][0]
lat, lon = coords[0], coords[1]

In [ ]:
start_date = '2020-01-01'
end_date = '2020-05-31'

In [ ]:
bbox = bbox_from_point(lat=lat, lon=lon, r=RADIUS) # WGS84 coordinates

In [ ]:
IPython.display.GeoJSON(shapely.geometry.box(*bbox).__geo_interface__)

## 3. Configure + Load + Preprocess Data Cube

In [ ]:
cube_config = CubeConfig(dataset_name='S2L1C',
                         band_names=['B03', 'B08', 'CLP'], # GREEN + NIR + CLP bands
                         tile_size=[2*RADIUS//10, 2*RADIUS//10],
                         geometry=bbox,
                         time_range=[start_date, end_date],
                         time_period='5D', ##### Set to 1D
                        )

In [ ]:
cube = open_cube(cube_config)
#cube # Verify the content of the cube

In [ ]:
# Drop Nans & Keep cloudless imagery, Convert to tensor (NIR + BG_NDWI) and metadata.
x, timestamps = cube2tensor(cube, max_cloud_proba=0.2, nans_how='any', verbose=1, plot_NDWI=False)

In [ ]:
#cube

## 4. Test Model

In [ ]:
# load pretrained model
model = Model(input_dim=2, hidden_dim=16, kernel_size=3, device='cpu', version='v0.0.1')
checkpoint_file = os.path.join(checkpoint_dir, model.folder, 'model.pth')
model.load_checkpoint(checkpoint_file=checkpoint_file)
model = model.eval()

In [ ]:
# Chunk image, detect boats and aggregate counts!
heatmaps, counts = model.chip_and_count(x, water_ndwi=0.5, plot_heatmap=True, timestamps=timestamps, max_frames=8, plot_indicator=True)

In [ ]:
##### Save AOI, timestamps, counts to geodB

meta_data = 'lat={}/lon={}/r={}'.format(lat, lon, RADIUS)
output = {}
for timestamp, count in zip(timestamps, counts):
    output[timestamp] = float(count)

print(meta_data)
for k,v in output.items():
    print(k,v)

In [ ]:
##### Automate analysis for AOI + time window